# Setup Environment

Import required packages:

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import os, cv2, json
from matplotlib import pyplot as plt

from detectron2 import model_zoo
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data.samplers.distributed_sampler import TrainingSampler
from detectron2.data import MetadataCatalog

Global variables and settings:

In [ ]:
path_model_dir = "/thecube/students/jravagli/outputs/detectron-12-epochs"
path_model_weights = os.path.join(path_model_dir, "model_final.pth")
# path_model_config = os.path.join(path_model_dir, "config.yaml")

path_ds_test_dir = "/thecube/students/jravagli/datasets/test"
path_ds_test_imgs = os.path.join(path_ds_test_dir, "image")
path_ds_test_json = os.path.join(path_ds_test_dir, "test.json")

path_output_dir = "/thecube/students/jravagli/outputs/detectron-eval"
path_res_json = os.path.join(path_output_dir, "results.json")

# Model settings
n_classes = 13 # Number of classes of the training set

# Load data and model

Register the test dataset:

In [ ]:
register_coco_instances("deepfashion_test", {}, path_ds_test_json, path_ds_test_imgs)

Load the trained model:

In [ ]:
# Use the same configuration used for training, with some changes for the 
# evaluation
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
# cfg.merge_from_file(path_model_config)
cfg.DATASETS.TRAIN = ()
cfg.DATASETS.TEST = ("deepfashion_test",)
cfg.DATALOADER.NUM_WORKERS = 8
cfg.MODEL.WEIGHTS = path_model_weights
cfg.SOLVER.IMS_PER_BATCH = 8
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128  # RoI batch size
cfg.MODEL.ROI_HEADS.NUM_CLASSES = n_classes
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # Confidence threshold for predictions (as in the DeepFashion2 paper)
cfg.OUTPUT_DIR = path_output_dir

predictor = DefaultPredictor(cfg)

# Quantitative Results

Evaluate the model on the test set:

In [ ]:
# Create the data loader
test_loader = build_detection_test_loader(cfg, "deepfashion_test")
# Create the evaluator
evaluator = COCOEvaluator("deepfashion_test", ("bbox", "segm"), False, output_dir=path_output_dir)
# Evaluate
dict_res = inference_on_dataset(predictor.model, test_loader, evaluator)

Save results:

In [ ]:
with open(path_res_json, 'w') as f:
    f.write(json.dumps(dict_res))
print("Saved")

# Qualitative Analysis

Create the data loader:

In [ ]:
list_dataset_dicts = DatasetCatalog.get("deepfashion_test")
metadata = MetadataCatalog.get("deepfashion_test")

Show network predictions on random samples:

In [ ]:
n_images = 2

sample_index = 0

for i in range(n_images):
    sample = list_dataset_dicts[sample_index]
    for ann in sample["annotations"]:
        ann.pop("keypoints", None)
    
    im = cv2.imread(sample["file_name"])
    
    # Draw ground truth
    v = Visualizer(im[:, :, ::-1], metadata)
    out_gt = v.draw_dataset_dict(sample)
    # Draw predictions
    v = Visualizer(im[:, :, ::-1], metadata)
    out_pred = v.draw_instance_predictions(predictor(im)["instances"].to("cpu"))
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,15))
    ax1.set_title("Ground Truth")
    ax1.imshow(out_gt.get_image())
    ax2.set_title("Prediction")
    ax2.imshow(out_pred.get_image())
    
    sample_index +=1